In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import polars as pl
import numpy as np
import seaborn as sns
import sklearn as sk
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from joblib import Parallel, delayed
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
import plotly.graph_objs as go
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
leap_raw_df = pl.scan_csv(f'/kaggle/input/leap-atmospheric-physics-ai-climsim/train.csv')
fetch_size =1000
input_columns = leap_raw_df.columns[1:557]
output_columns = leap_raw_df.columns[557:924]
X = leap_raw_df.select(pl.col(input_columns)).fetch(fetch_size)
Y = leap_raw_df.select(pl.col(output_columns)).fetch(fetch_size)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
constant_epson = 1
constant_epson_r = 1
def log_transform(x):
    return (np.log1p(x+constant_epson) +constant_epson_r)

# Convert Polars DataFrame to NumPy array
X_np = X.to_numpy()
Y_np = Y.to_numpy()

transformer_X = StandardScaler().fit(X_np)
transformer_Y = StandardScaler().fit(Y_np)

X_transformed = transformer_X.transform(X_np)
Y_transformed = transformer_Y.transform(Y_np)



# Log
#transformer =FunctionTransformer(log_transform)
#X_transformado=transformer.transform(X_np)
#Y_transformado=transformer.transform(Y_np)
del X_np,Y_np,X,Y
print(X_transformed.dtype)
print(Y_transformed.dtype)

In [ ]:
X_transformed.shape[1]

In [ ]:
def grafico_performace(feature_weighted_fitness):
    
    # Create an interactive plot
    features = list(range(1, len(feature_weighted_fitness) + 1))
    trace = go.Scatter(
        x=features,
        y=feature_weighted_fitness,
        mode='lines+markers',
        marker=dict(size=10, color='red'),
        line=dict(color='blue', width=2),
        hoverinfo='x+y+text',
        text=['Feature {}'.format(i) for i in features]
    )

    layout = go.Layout(
        title='Feature Frequency*Fitness Over Generations',
        xaxis=dict(title='Feature'),
        yaxis=dict(title='Frequency*Fitness'),
        hovermode='closest'
    )

    fig = go.Figure(data=[trace], layout=layout)
    fig.show()

In [ ]:
import cupy as cp

In [ ]:
def fitness_function(solution, X_transformed =X_transformed, Y_transformed=Y_transformed):
    from statistics import mean
    selected_features = np.where(solution == 1)[0]
    if len(selected_features) == 0:
        return 0
    X_selected = X_transformed[:, selected_features]
    X_train, X_val, Y_train, Y_val = train_test_split(X_selected, Y_transformed, test_size=0.2, random_state=42)
    X_train=cp.asarray(X_train)
    X_val=cp.asarray(X_val)
    Y_train=cp.asarray(Y_train)
    Y_val=cp.asarray(Y_val)
    r2_values = []
    
    for i in range(361,362):
        model = XGBRegressor(
            n_estimators= 1150,
            max_depth=56,
            learning_rate=0.003,
            colsample_bytree=0.9,
            subsample=0.8,
            min_child_weight=1,
            random_state=42,
            objective='reg:squarederror',
            n_jobs=-1,  # Use all available cores
            tree_method='hist',
            device ='cuda',
            early_stopping_rounds = 6
        )
        zeros =0
        model.fit(
            X_train, Y_train[:, i],
            eval_set=[(X_val, Y_val[:, i])],
            verbose=False
        )
        y_pred = model.predict(X_val)
        r2 = r2_score(Y_val[:, i].get(), y_pred)
        if r2 == 0:
            r2 = 1
        r2_values.append(r2)
    punicao = 3.96e-4  # 0.1 R^2 para metade do dataset
    print(f"individo incial com r^2 de {mean(r2_values)}, e com score punido de {mean(r2_values) - punicao * X_selected.shape[1]}, dimensão={X_selected.shape[1]}")
    return -mean(r2_values) +punicao*X_selected.shape[1]

In [ ]:


# k_factor function
def k_factor(kf):
    return 1 if random.random() < kf else 0

def gaining_phase(population, fitness_tuples, kf):
    new_population = population.copy()
    dimensao = 60
    f_unidimensionais = 16
    jota = 355  # Starting index for unidimensional features

    for i in range(len(population)):
        # Find the junior_xt_position
        junior_xt_position = int(next(idx for idx, fit in fitness_tuples if idx == i))
        junior_xt_0 = int((junior_xt_position - 1) % len(fitness_tuples))
        junior_xt_1 = int((junior_xt_position + 1) % len(fitness_tuples))

        partner = int(random.choice(range(len(population))))

        if (fitness_tuples[junior_xt_position][1] > fitness_tuples[partner][1])and k_factor(kf):
            for j in range(25):
                if j < 6:
                    start = j * dimensao
                    end = start + dimensao
                elif j > 21:
                    start = (360 + f_unidimensionais) +(j-22)*dimensao
                    end = start + dimensao
                else:
                    start = (jota + j) 
                    end = start+1                  
                    
                new_population[i][start:end] = (
                    new_population[i][start:end] +
                    k_factor(kf) * (
                        population[partner][start:end] - 
                        new_population[i][start:end] + 
                        population[fitness_tuples[junior_xt_0][0]][start:end] - 
                        population[int(fitness_tuples[junior_xt_1][0])][start:end]
                    )
                )
                # Ensure the new solution values are between 0 and 1
                new_population[i][start:end] = np.clip(new_population[i][start:end], 0, 1)
                
            for j in range(25):
                if j < 6:
                    start = j * dimensao
                    end = start + dimensao
                elif j > 21:
                    start = (360 + f_unidimensionais) +(j-22)*dimensao
                    end = start + dimensao
                else:
                    start = (jota + j) 
                    end = start+1  

                new_population[i][start:end] = (
                    new_population[i][start:end] +
                    k_factor(kf) * (
                        new_population[i][start:end] - 
                        population[partner][start:end] + 
                        population[fitness_tuples[junior_xt_0][0]][start:end] - 
                        population[int(fitness_tuples[junior_xt_1][0])][start:end]
                    )
                )
                new_population[i][start:end] = np.clip(new_population[i][start:end], 0, 1)

    return new_population



# Senior gaining phase function
def senior_gaining_phase(population, fitness_tuples, kf, p=0.1):
    num_individuals = len(population)
    num_seniors = int(num_individuals * p)
    num_juniors = num_seniors
    num_middles = num_individuals - num_seniors - num_juniors

    # Ensure at least one senior, middle, and junior
    if num_seniors == 0:
        num_seniors = 1
    if num_juniors == 0:
        num_juniors = 1
    if num_middles < 0:
        num_middles = num_individuals - 2

    senior_indices = [idx for idx, fit in fitness_tuples[:num_seniors]]
    middle_indices = [idx for idx, fit in fitness_tuples[num_seniors:num_seniors + num_middles]]
    junior_indices = [idx for idx, fit in fitness_tuples[-num_juniors:]]

    new_population = population.copy()
    dimensao = 60
    f_unidimensionais = 16
    jota = 355  # Starting index for unidimensional features

    for i in range(len(population)):
        senior_index = random.choice(senior_indices)
        middle_index = random.choice(middle_indices)
        junior_index = random.choice(junior_indices)

        if fitness_tuples[i][1] > fitness_tuples[middle_index][1]:
            for j in range(25):
                if j < 6:
                    start = j * dimensao
                    end = start + dimensao
                elif j > 21:
                    start = (360 + f_unidimensionais) +(j-22)*dimensao
                    end = start + dimensao
                else:
                    start = (jota + j) 
                    end = start+1  


                new_population[i][start:end] = (
                    new_population[i][start:end] +
                    k_factor(kf) * (
                        population[senior_index][start:end] - 
                        population[junior_index][start:end] + 
                        population[middle_index][start:end] - 
                        new_population[i][start:end]
                    )
                )
                # Ensure the new solution values are between 0 and 1
                new_population[i][start:end] = np.clip(new_population[i][start:end], 0, 1)
            for j in range(25):
                if j < 6:
                    start = j * dimensao
                    end = start + dimensao
                elif j > 21:
                    start = (360 + f_unidimensionais) +(j-22)*dimensao
                    end = start + dimensao
                else:
                    start = (jota + j - 16) 
                    end = start+1

                new_population[i][start:end] = (
                    new_population[i][start:end] +
                    k_factor(kf) * (
                        population[senior_index][start:end] - 
                        population[junior_index][start:end] + 
                        new_population[i][start:end] - 
                        population[middle_index][start:end]
                    )
                )
                new_population[i][start:end] = np.clip(new_population[i][start:end], 0, 1)

    return new_population

# Initialize the population
def initialize_population(pop_size, num_features=25, max_features=4):
    dimensao = 60
    size_total = 556#numero total de features
    population = np.ones((pop_size, size_total))
    f_unidimensionais =16
    jota = 355 
    for i in range(pop_size):
        features_usadas = num_features
        while features_usadas > max_features:
            for k in range(num_features):
                j = random.randint(0, 24)
                if random.randint(0, 100) > 50:
                    if j < 6:
                        population[i][j * dimensao:(j * dimensao + dimensao)] = 0
                        features_usadas -= 1
                    if j >21:
                        #features unidimensionais
                        start = (j-f_unidimensionais)*dimensao + f_unidimensionais
                        end = start + dimensao
                        population[i][start:end] = 0
                        features_usadas -= 1
                    else:
                        start =jota+j
                        population[i][jota + j] = 0
                        features_usadas -= 1
                        
                    # Stop the loop if features_usadas <= max_features
                    if features_usadas <= max_features:
                        break
    return population


# Re-evaluate fitness function
def revaluate_fitness(population, X, y):
    fitness_values = [fitness_function(individual, X, y) for individual in population]
    fitness_tuples = sorted(enumerate(fitness_values), key=lambda x: x[1])
    return fitness_tuples

def population_reduction(population, fitness_tuples, generation_atual, generation_max,pop_size_inicial,low_b=0.12,high_b=0.6):
    if int(pop_size_inicial*low_b) < len(population):
        np_min =pop_size_inicial*low_b
        np_max =pop_size_inicial*high_b
        np_new = int((np_min - np_max) * (generation_atual / generation_max) + np_max -1)

        for _ in range(len(population) - np_new):
            remove_idx = int(fitness_tuples[-1][0])
            population = np.delete(population, remove_idx, axis=0)
            fitness_tuples = fitness_tuples[:-1]

            # Update the remaining indices in fitness_tuples
            fitness_tuples = [(idx if idx < remove_idx else idx - 1, fit) for idx, fit in fitness_tuples]
            fitness_tuples.sort(key=lambda x: x[1])
    
    return population, fitness_tuples

# Initialize feature tracking

import numpy as np
import plotly.graph_objs as go

def FS_pBGSK(X, y, pop_size=100, num_generations=50, num_jobs=-1, kf=0.999):
    num_features = X.shape[1]
    population = initialize_population(pop_size, num_features, round(num_features - 5))
    fitness_tuples = revaluate_fitness(population, X, y)
    best_fitness_overall = float('inf')
    feature_usage = np.zeros(X.shape[1])
    feature_weighted_fitness = np.zeros(X.shape[1])
    #avaliação inicial
    for individual, fitness in zip(population, [fit[1] for fit in fitness_tuples]):
        feature_usage += individual
        feature_weighted_fitness += individual * fitness
    
    grafico_performace(feature_weighted_fitness)    
    for generation in range(num_generations):
        population = gaining_phase(population, fitness_tuples, kf)
        population = senior_gaining_phase(population, fitness_tuples, kf)

        best_index_g, best_fitness_g = fitness_tuples[0]
        if best_fitness_overall > best_fitness_g:
            best_fitness_overall = best_fitness_g
            best_solution = population[best_index_g]
            print(f"New Best Overall {generation + 1}: Best Fitness = {-best_fitness_overall}, Best Solution = {best_solution}, Dimension = {best_solution.sum()}")

        population, fitness_tuples = population_reduction(population, fitness_tuples, generation + 1, num_generations, pop_size)
        print(f"Current population shape: {population.shape}")

    # Final evaluation to return the best solution overall
    fitness_tuples = revaluate_fitness(population, X, y)
    best_index, best_fitness = fitness_tuples[0]
    best_solution = population[best_index]
    print(f"Checking best performance: {fitness_function(best_solution, X, y)}, Dimension = {best_solution.sum()}")
    # Normalize weighted fitness by feature usage
    feature_weighted_fitness = np.divide(feature_weighted_fitness, feature_usage, out=np.zeros_like(feature_weighted_fitness), where=feature_usage != 0)

    return best_solution, best_fitness_overall, feature_weighted_fitness



In [ ]:
import plotly.graph_objs as go
best_solution, best_fitness_overall, feature_weighted_fitness = FS_pBGSK(X_transformed, Y_transformed)

grafico_performace(feature_weighted_fitness)
# Create an interactive plot for the single best solution
selected_features = np.where(best_solution == 1)[0]
trace = go.Scatter(
    x=selected_features,
    y=np.ones_like(selected_features),  # All y-values are 1, as we are only showing selected features
    mode='markers',
    marker=dict(size=10, color='red'),
    hoverinfo='x+text',
    text=['Feature {}'.format(i) for i in selected_features]
)

layout = go.Layout(
    title='Selected Features in the Best Solution',
    xaxis=dict(title='Feature Index'),
    yaxis=dict(title='', showticklabels=False),  # Hide y-axis labels
    hovermode='closest'
)

fig = go.Figure(data=[trace], layout=layout)
fig.show()